# Estimation and Analysis of the ground-truth OD matrix

#### adding required packages

In [104]:
import pandas as pd
import numpy as np
import pickle
import time
# Install a conda package in the current Jupyter kernel
import sys
# !conda install --yes --prefix {sys.prefix} pyproj
# import sys
# !{sys.executable} -m pip install geopandas

# !pip uninstall shapely
# !conda install -c conda-forge shapely
# !pip uninstall pyproj
# !conda install -c conda-forge pyproj
from pyproj import Proj, transform
from shapely.geometry import Point
import geopandas as gpd

#### specifying the saving location 

In [105]:
savingLoc = "Y:/ZahraEftekhar/phase4/"

#### functions for proximity analysis

Here, we create classes and functions to automate the proximity analysis. These analyses include changing the coordinate system, mapping the location coordinates to their associated TAZ (traffic analysis zone).

In [106]:
#this class change coordinate system and map location coordinates to zones:
class LongLat:
    def __init__(self, *args):
        self.TAZ = 0
    def set_location(self, x, y):
        from shapely.geometry import Point
        self.location = Point(x, y)

    def changeCoordSys(self, initial: str = 'epsg:23031', final: str = 'epsg:28992'):
        from pyproj import Proj, transform
        from shapely.geometry import Point
        self.location = Point(transform(Proj(init=initial), Proj(init=final), self.location.x, self.location.y))

    def zoneMapping(self, onepolygon, polygonName):
        if (onepolygon.contains(self.location)):
            self.TAZ = polygonName
# this class only reads SHP:
class TAZmap:
    def __init__(self): pass

    def set_map(self, value):
        import geopandas as gpd
        self.map = gpd.read_file(value)

#### building the zero OD Matrix based on the number of zones in the SHP

In [107]:
# building the zero OD Matrix based on the number of zones in the SHP
map_mzr = TAZmap()
map_mzr.set_map("{a}amsterdamMezuroZones.shp".format(a=savingLoc))
inputs = map_mzr.map.geometry
amsterdamMezuroZones = pd.read_csv("{a}amsterdamMezuroZones.CSV".format(a=savingLoc), usecols=['mzr_id'])
tazNames = amsterdamMezuroZones['mzr_id']
zoneZero = pd.Series(0)
matrixRowColNames = tuple(zoneZero.append(tazNames))
odsize=len(matrixRowColNames)
del map_mzr
ODMatrix_df = pd.DataFrame(np.zeros((odsize, odsize), dtype=np.int32), columns=matrixRowColNames,
                           index=matrixRowColNames)  # creating empty OD matrix

## Specifying the OD matrix estimation interval
=========================================================

In [108]:
ODstart = "6:30:00"
ODend = "9:30:00"
startTime_OD = pd.to_timedelta(ODstart)
endTime_OD = pd.to_timedelta(ODend)

#### Reading the travel diaries to estimate the OD matrix

In [109]:
with open('{a}1.trueLocExperienced.pickle'.format(a=savingLoc), 'rb') as handle:
    itemlistExperienced = pickle.load(handle)

======================================================================================================

#### estimating the ground-truth OD matrix

In [102]:
#we time the process
startTime = time.time()
for ID in itemlistExperienced.keys():#itemlistExperienced.keys() or ['100158'] or ['100048']
    activityListExperienced = itemlistExperienced[ID]
    activityListExperienced.loc[:,"end"].iloc[-1] = pd.to_timedelta(activityListExperienced.loc[:,"end"].iloc[-1]
                                                                  )+ pd.to_timedelta('24:00:00')
    j=1
    while j < len(np.arange(len(activityListExperienced))):
        if j==len(np.arange(len(activityListExperienced)))-1:
            start_time1 = pd.to_timedelta((activityListExperienced.loc[:,"start"]).iloc[j])
            end_time1 = pd.to_timedelta((activityListExperienced.loc[:,"end"]).iloc[j])
            start_time2 = pd.to_timedelta((activityListExperienced.loc[:,"start"]).iloc[0])+ pd.to_timedelta('24:00:00')
            endActivity = end_time1
            startNewActivity = start_time2
            if pd.to_timedelta('23:59:59')>=pd.to_timedelta(ODstart)>=pd.to_timedelta(start_time1):
                startTime_OD = pd.to_timedelta(ODstart)
            else:
                startTime_OD = pd.to_timedelta(ODstart) + pd.to_timedelta('24:00:00')
            if pd.to_timedelta('23:59:59')>= pd.to_timedelta(ODend) >=pd.to_timedelta(start_time1):
                endTime_OD =pd.to_timedelta(ODend)
            else:
                endTime_OD = pd.to_timedelta(ODend)+ pd.to_timedelta('24:00:00')
        else:
            start_time1 = pd.to_timedelta(activityListExperienced.loc[:,"start"].iloc[j])
            end_time1 = pd.to_timedelta(activityListExperienced.loc[:,"end"].iloc[j])
            start_time2 = pd.to_timedelta(activityListExperienced.loc[:,"start"].iloc[j+1])
            endActivity = end_time1 
            startNewActivity = start_time2 
        if pd.to_timedelta(start_time1) <= pd.to_timedelta(startTime_OD) < pd.to_timedelta(startNewActivity):
            if endTime_OD <= endActivity:
                break
            else:
                while pd.to_timedelta(endTime_OD) > pd.to_timedelta(endActivity):
                    point1 = LongLat()
                    point1.set_location(x=float(activityListExperienced.loc[:,"x"].iloc[j]),
                                        y=float(activityListExperienced.loc[:,"y"].iloc[j]))#*********ghablan:j-1
                    point1.changeCoordSys()
                    for k in range(len(tazNames)):
                        point1.zoneMapping(inputs[k], tazNames[k])
                    origin = point1.TAZ
#                     print(activityListExperienced.loc[:,"VEHICLE"].iloc[0])
                    point2 = LongLat()
                    if j == len(activityListExperienced) - 1:
                        point2.set_location(x=float(activityListExperienced.loc[:,"x"].iloc[0]),
                                            y=float(activityListExperienced.loc[:,"y"].iloc[0]))
                    else:
                        point2.set_location(x=float(activityListExperienced.loc[:,"x"].iloc[j+1]),
                                        y=float(activityListExperienced.loc[:,"y"].iloc[j+1])) #*****ghablan: j
                    point2.changeCoordSys()
                    for k in range(len(tazNames)):
                        point2.zoneMapping(inputs[k], tazNames[k])
                    destination = point2.TAZ
                    ODMatrix_df[origin][destination] = ODMatrix_df[origin][destination] + 1
                    j += 1
                    if j > len(np.arange(len(activityListExperienced))) - 1: break
                    if j== len(np.arange(len(activityListExperienced)))-1:
                        end_time1 = pd.to_timedelta((activityListExperienced.loc[:,"end"]).iloc[0])+ pd.to_timedelta('24:00:00')
                    else:
                        end_time1 = pd.to_timedelta((activityListExperienced.loc[:,"end"]).iloc[j])
                    endActivity = end_time1 # when using inconsistent timings :endActivity = min(end_time1, start_time2)
                break
            # continue

        j += 1

print((time.time() - startTime)//60,'minutess')

1
100007
100009
10001
100014
10006
100083
100086
100099
100101
100110
100112
100144
10015
100158
100182
100197
100198
10021
100216
100222
100238
100303
100305
100336
100339
100361
100381
10039
100409
0.0 minutess


In [103]:
print(np.sum(np.sum(ODMatrix_df, axis=0)))

36
